In [4]:
!pip -q install redis redis-cli

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

## Common Redis Use Cases

> **Document scope**: Caching; Session Store; Leaderboards; Message Processing & Delivery; Indexing & Querying.

---

### Use Cases (p. 2)

1. Caching  
2. Session Store  
3. Leaderboards  
4. Message Processing and Delivery  
5. Indexing and Querying  :contentReference[oaicite:1]{index=1}

---

### Caching (pp. 3–10)

#### Why add a cache? (pp. 4–5)

**Motivation.** Disk-based databases introduce latency and scale-up costs under heavy read workloads. Adding an in-memory Redis layer yields sub‑millisecond latency and high throughput while simplifying data access. **What to cache:** database results/HTTP responses, key–value pairs, native Redis structures, and even binary files. :contentReference[oaicite:2]{index=2}

> **Sidenote**
>
> [Link: Caching Best Practices (AWS)](https://aws.amazon.com/caching/best-practices/)  
> **Concept**: `Cache effectiveness & safety` → Criteria for what/when to cache.  
> **Context**: Identify data with acceptable staleness and repeat access.  
> **Example**: Cache product details with minute-level tolerance.  
> **Implication**: Better hit ratio and lower origin load. :contentReference[oaicite:3]{index=3}

---

#### Caching Pattern — Cache-Aside / Look‑Aside (p. 6)

**Definition.** Application reads through Redis first; on miss, load from DB, then populate cache. Writes go directly to DB; cache entries are invalidated or updated by the app. **Pros:** fault tolerant, cache holds only hot data. **Cons:** extra network hops on miss, potential staleness until refresh. :contentReference[oaicite:4]{index=4}

> **Sidenote**
>
> [Link: Cache-Aside pattern (Microsoft)](https://learn.microsoft.com/en-us/azure/architecture/patterns/cache-aside)  
> **Concept**: `Cache-Aside` → On-demand population.  
> **Context**: General-purpose pattern for read-heavy systems.  
> **Example**: `GET user:42` → miss → DB → cache.  
> **Implication**: Simple operational model; application owns consistency. :contentReference[oaicite:5]{index=5}

```python
# Cache-aside read: try Redis, fall back to DB, then populate Redis
val = r.execute_command("GET", "user:42")
if val is None:
    # app/db_client.get_user(42) -> {"id": 42, "name": "Ana"}
    r.execute_command("SET", "user:42", '{"id":42,"name":"Ana"}')
print("OK - cached after miss")
````

> **Sidenote**
>
> [Link: TTL](https://redis.io/docs/latest/commands/ttl/)
> **Command**: `TTL` → Seconds to live for a key (−2=missing, −1=no expire).
> **Pattern**: `TTL key`
> **Example**: `TTL user:42`
> **Result**: Integer seconds remaining or negative sentinel. ([Redis][1])

---

#### Caching Pattern — Write‑Through with RedisGears RGSync (p. 7)

**Definition.** Application writes via Redis; RedisGears synchronously writes through to the primary DB. **Pros:** fewer hops on reads, data stays in sync, simple for app. **Cons:** write path overhead; cache must hold full working set; often pre‑warm/populate.&#x20;

> **Sidenote**
>
> [Link: RedisGears RGSync (write‑through/behind)](https://redis.io/docs/latest/operate/oss_and_stack/stack-with-enterprise/gears-v1/python/recipes/write-behind/)
> **Concept**: `RGSync` → RedisGears recipe for syncing Redis ↔ RDBMS.
> **Context**: Write-through/behind for caches with strong read SLAs.
> **Example**: Map `hash user:*` to `users` table.
> **Implication**: App logic simplified; data flows governed by Gears. ([Redis][2])

```python
# Write-through-ish: update cache; Gears (RGSync) writes to DB
r.execute_command("HSET", "user:42", "name", "Ana")
print("OK - Redis updated; RGSync handles DB")
```

> **Sidenote**
>
> [Link: HSET](https://redis.io/docs/latest/commands/hset/)
> **Command**: `HSET` → Set field(s) in a hash.
> **Pattern**: `HSET key field value [field value ...]`
> **Example**: `HSET user:42 name "Ana"`
> **Result**: Integer: new fields added. ([Redis][3])

---

#### Caching Pattern — Write‑Behind with RedisGears RGWriteBehind (p. 8)

**Definition.** App writes to Redis; RedisGears stages changes (e.g., via Streams) and asynchronously persists them to the DB. **Pros:** fast writes; fewer network hops; cache always current. **Cons:** backend lags (eventual consistency); cache must contain full working set.&#x20;

> **Sidenote**
>
> [Link: RGWriteBehind (GitHub)](https://github.com/RedisGears/rgsync)
> **Concept**: `Write‑Behind` → Async DB persistence via Gears.
> **Context**: High‑write, read‑heavy systems tolerant to eventual consistency.
> **Example**: Mirror `person:*` hashes to `persons` table.
> **Implication**: Absorbs write spikes; requires recovery strategy. ([GitHub][4])

```python
# Write-behind: fast path writes to Redis; background pipeline persists
r.execute_command("HSET", "order:9001", "status", "PAID")
print("OK - staged for async DB persistence")
```

---

#### Caching Pattern — Read‑Through with RedisGears (p. 9)

**Definition.** Reads are served from Redis; on miss, a Gears function fetches from the DB and populates Redis before returning. **Pros:** app simplicity; cache updated only when needed. **Cons:** miss path cost; staleness must be considered. (Not an official canned recipe in the slide.)&#x20;

```python
# Read-through sketch: on miss, a server-side function fetches & fills
val = r.execute_command("GET", "product:sku:Z9")
if val is None:
    # server-side logic (Gears/Function) would populate before returning
    r.execute_command("SET", "product:sku:Z9", '{"sku":"Z9"}')
print("OK - served from Redis")
```

---

#### Caching Pattern — CDC Cache Replica (p. 10)

**Definition.** Data is written to the system of record; change data capture (CDC) replicates changes into Redis to keep a read cache warm and current. **Pros:** warm cache; avoids staleness after CDC applies; simple for reads. **Cons:** cache miss possible before replication; cache must hold relevant data.&#x20;

> **Sidenote**
>
> [Link: CDC (SQL Server)](https://learn.microsoft.com/en-us/sql/relational-databases/track-changes/about-change-data-capture-sql-server?view=sql-server-ver17)
> **Concept**: `Change Data Capture` → Log-based replication of inserts/updates/deletes.
> **Context**: Feed Redis read cache from DB logs.
> **Example**: Replicate `orders` CDC into `order:*` keys.
> **Implication**: Near-real-time cache consistency without app writes. ([Microsoft Learn][5])

---

#### Expiration & freshness controls (cross‑cutting)

```python
# Set with expiration (seconds)
r.execute_command("SETEX", "user:42:profile", 300, '{"id":42,"n":"Ana"}')
print("OK - expires in 300s")
```

> **Sidenote**
>
> [Link: SETEX](https://redis.io/docs/latest/commands/setex/)
> **Command**: `SETEX` → Set value and TTL atomically.
> **Pattern**: `SETEX key seconds value`
> **Example**: `SETEX user:42:profile 300 {...}`
> **Result**: `OK`. ([Redis][1])

```python
# Read while updating expiry
val = r.execute_command("GETEX", "user:42:profile", "EX", 300)
print("Value and refreshed TTL")
```

> **Sidenote**
>
> [Link: GETEX](https://redis.io/docs/latest/commands/getex/)
> **Command**: `GETEX` → Get the value and set/alter TTL.
> **Pattern**: `GETEX key [EX seconds|PX ms|EXAT ts|PXAT ms|PERSIST]`
> **Example**: `GETEX k EX 60`
> **Result**: Bulk string (value) or nil. ([Redis][1])

```python
# Inspect & control key lifetime
r.execute_command("EXPIRE", "user:42:profile", 60)
r.execute_command("TTL", "user:42:profile")
r.execute_command("PERSIST", "user:42:profile")
print("OK - TTL set/read/cleared")
```

> **Sidenote**
>
> [Link: EXPIRE](https://redis.io/docs/latest/commands/expire/) • [Link: TTL](https://redis.io/docs/latest/commands/ttl/) • [Link: PERSIST](https://redis.io/docs/latest/commands/persist/)
> **Command**: `EXPIRE` sets TTL; `TTL` reads TTL (`-2` missing, `-1` no TTL); `PERSIST` clears TTL.
> **Pattern**: `EXPIRE key seconds` • `TTL key` • `PERSIST key`
> **Result**: Integers or `OK`. ([Redis][1])

---

### Session Store (pp. 11–14)

#### Overview & differences from “just cache” (pp. 12–13)

**Key properties.** Session data must be **fast**, **always available while active**, and may require **persistence**/HA for recovery. Unlike a generic cache, active session data in Redis is the **source of truth**, evolves over time, and typically has stronger HA/persistence requirements.&#x20;

> **Sidenote**
>
> [Link: Redis persistence](https://redis.io/docs/latest/operate/oss_and_stack/management/persistence/)
> **Concept**: `RDB/AOF` → Durability for session recovery.
> **Context**: Session stores often enable AOF (multi-part in Redis 7+) for minimal loss.
> **Example**: Combine replicas + AOF.
> **Implication**: Survive process/node restarts with minimal session loss. ([Redis][6])

> **Sidenote**
>
> [Link: Sentinel](https://redis.io/docs/latest/operate/oss_and_stack/management/sentinel/) • [Link: Cluster](https://redis.io/docs/latest/operate/oss_and_stack/management/scaling/)
> **Concept**: `HA & scale-out` → Failover (Sentinel) and sharding (Cluster).
> **Context**: Keep sessions available; scale horizontally for load.
> **Implication**: Continuous availability and predictable throughput. ([Redis][7])

#### Common session data (p. 14)

| Use Case      | Data Stored                                                       | Features             |   |
| ------------- | ----------------------------------------------------------------- | -------------------- | - |
| Login State   | last access; source (browser/app); type (long‑term/browser-based) | TTL; HA; persistence |   |
| Navigation    | breadcrumb trail                                                  | HA                   |   |
| Shopping Cart | items; last update                                                | HA; persistence      |   |
| API Tokens    | app ID; scopes; start time                                        | TTL; HA              |   |

```python
# Token with TTL (session) + quick introspection
r.execute_command("SETEX", "sess:token:abc", 1800, '{"uid":42,"scopes":["read"]}')
ttl = r.execute_command("TTL", "sess:token:abc")
print(f"TTL={ttl}s")
```

---

### Leaderboards (pp. 15–17)

**Requirements.** High‑write (frequent score updates) + high‑read (ranking views) at large scale. **Why Redis.** Sorted sets are an in‑memory, automatically ordered structure with O(log N) updates and O(log N + M) range queries. Ties (equal scores) are ordered lexicographically by member.&#x20;

> **Sidenote**
>
> [Link: Sorted sets](https://redis.io/docs/latest/develop/data-types/sorted-sets/)
> **Concept**: `ZSET` → Members ordered by floating‑point score; tie‑break by lexicographic member.
> **Context**: Real‑time leaderboards.
> **Example**: Rank users by points.
> **Implication**: Efficient inserts, rank queries, and top‑K retrieval. ([Redis][8])

```python
# Add/update scores
r.execute_command("ZADD", "lb:game", 100, "u:alice", 95, "u:bob")
r.execute_command("ZINCRBY", "lb:game", 10, "u:bob")
print("OK - scores upserted")

# Get top 3 with scores (high to low)
top3 = r.execute_command("ZREVRANGE", "lb:game", 0, 2, "WITHSCORES")
print(top3)
```

> **Sidenote**
>
> [Link: ZADD](https://redis.io/docs/latest/commands/zadd/) • [Link: ZINCRBY](https://redis.io/docs/latest/commands/zincrby/) • [Link: ZREVRANGE](https://redis.io/docs/latest/commands/zrevrange/)
> **Command**: Sorted‑set upsert/increment/range.
> **Pattern**: `ZADD key score member [score member ...]` • `ZINCRBY key inc member` • `ZREVRANGE key start stop [WITHSCORES]`
> **Result**: Integers (`ZADD`), new score (`ZINCRBY`), arrays (`ZREVRANGE`). ([Redis][9])

---

### Message Processing and Delivery (pp. 18–23)

#### Message Queues with Lists (p. 20)

**Characteristics.** FIFO, batching/storage until consumed; one consumer dequeues at a time; pull‑based; **at‑least‑once** delivery.&#x20;

```python
# Producer: enqueue to the tail
r.execute_command("RPUSH", "q:jobs", "job-1", "job-2")
print("OK - jobs queued")

# Consumer: blocking pop with timeout
job = r.execute_command("BRPOP", "q:jobs", 5)
print(job)  # e.g., ["q:jobs", "job-1"]
```

> **Sidenote**
>
> [Link: BRPOP](https://redis.io/docs/latest/commands/brpop/)
> **Command**: `BRPOP` → Blocking pop from list tail with timeout.
> **Pattern**: `BRPOP key [key ...] timeout`
> **Result**: `[key, element]` or `nil` on timeout. ([Redis][10])

> **Sidenote**
>
> [Link: LMOVE](https://redis.io/docs/latest/commands/lmove/)
> **Command**: `LMOVE` → Atomically move between lists (reliable queues).
> **Pattern**: `LMOVE src dst LEFT|RIGHT LEFT|RIGHT`
> **Result**: Moved element or `nil`. *(Preferred over legacy `RPOPLPUSH` for new designs.)* ([Redis][10])

#### Priority Queues with Sorted Sets (p. 21)

**Characteristics.** Ordered by priority (score); pull‑based; **at‑least‑once**; order defined by score (then lexicographically).&#x20;

```python
# Producer: lower score = higher priority
r.execute_command("ZADD", "q:prio", 1, "job:urgent", 10, "job:normal")
print("OK - prioritized")

# Consumer: pop the smallest (highest priority)
item = r.execute_command("ZPOPMIN", "q:prio")
print(item)  # e.g., ["job:urgent", "1"]
```

> **Sidenote**
>
> [Link: ZPOPMIN](https://redis.io/docs/latest/commands/zpopmin/)
> **Command**: `ZPOPMIN` → Remove & return the lowest‑score members.
> **Pattern**: `ZPOPMIN key [count]`
> **Result**: `[member, score, ...]`. ([Redis][11])

#### Publish/Subscribe (p. 22)

**Characteristics.** Fan‑out broadcast; push‑based; **at‑most‑once** (no persistence); order not guaranteed.&#x20;

```python
# Subscriber (channel "events")
# (Pseudo; in redis-py you'd use pubsub)
r.execute_command("SUBSCRIBE", "events")
print("listening...")

# Publisher
r.execute_command("PUBLISH", "events", "hello")
print("published")
```

> **Sidenote**
>
> [Link: Pub/Sub](https://redis.io/docs/latest/develop/pubsub/)
> **Concept**: `Pub/Sub` → Fire‑and‑forget broadcast to live subscribers.
> **Context**: Notifications and transient signals.
> **Implication**: No delivery if subscriber disconnected; use Streams for durability. ([Redis][12])

#### Streams (p. 23)

**Characteristics.** Append‑only log; ordered entries with IDs; consumer groups support horizontal consumption; **at‑least‑once** with acknowledgements; push/pull hybrid (XREAD / XREADGROUP).&#x20;

```python
# Producer: append to stream
r.execute_command("XADD", "events:sales", "*", "item", "A1", "qty", "2")
print("OK - appended")

# Consumer group: create & read
r.execute_command("XGROUP", "CREATE", "events:sales", "g1", "$", "MKSTREAM")
msgs = r.execute_command("XREADGROUP", "GROUP", "g1", "c1", "COUNT", 10, "STREAMS", "events:sales", ">")
print(msgs)

# Acknowledge
# (assume id is "1700000000000-0")
r.execute_command("XACK", "events:sales", "g1", "1700000000000-0")
print("OK - acked")
```

> **Sidenote**
>
> [Link: Streams](https://redis.io/docs/latest/develop/data-types/streams/)
> **Command set**: `XADD`, `XREAD`, `XGROUP`, `XREADGROUP`, `XACK`, `XPENDING`, etc.
> **Return values**: Arrays of `[stream, [[id, [field, value, ...]], ...]]`; acks/pending counts for group introspection.
> **Implication**: Durable log semantics with consumer groups; at‑least‑once unless you design idempotency. ([Redis][13])

---

### Indexing & Querying (pp. 24–30)

**Concept.** Index = alternate access path mapping a **secondary** attribute to a **primary key**. In Redis, you can build secondary indexes using core data structures (for numeric, lexicographic, hash, bitmap patterns) or an inverted index via the Redis Query Engine (RediSearch) for full‑text/JSON queries.&#x20;

---

#### Numerical Index — Sorted Sets (p. 26)

**Pattern.** Store entries as members with a numeric score; query by score ranges.

```python
# Index: price as score; id as member
r.execute_command("ZADD", "idx:price", 19.99, "item:1", 5.49, "item:2")

# Range: $5 to $20 (inclusive) with scores
rows = r.execute_command("ZRANGEBYSCORE", "idx:price", 5, 20, "WITHSCORES")
print(rows)
```

> **Sidenote**
>
> [Link: ZRANGE (BYSCORE) / ZRANGEBYSCORE](https://redis.io/docs/latest/commands/zrange/)
> **Command**: `ZRANGE ... BYSCORE` (new style) or `ZRANGEBYSCORE` (classic).
> **Pattern**: `ZRANGE key min max BYSCORE [REV] [LIMIT offset count] [WITHSCORES]`
> **Result**: Members (optionally with scores). ([Redis][14])

---

#### Lexicographical Index — Sorted Sets (p. 27)

**Pattern.** Insert all members with the **same score** to enable lexicographic ordering; query by lexical ranges.

```python
# Index: product names; all at score 0 for lex order
r.execute_command("ZADD", "idx:name", 0, "apple", 0, "applet", 0, "banana")

# Prefix query: names starting with "app"
res = r.execute_command("ZRANGEBYLEX", "idx:name", "[app", "[app\xff")
print(res)
```

> **Sidenote**
>
> [Link: ZRANGEBYLEX / ZLEXCOUNT](https://redis.io/docs/latest/commands/zrangebylex/)
> **Command**: Lex range over equal‑score ZSETs; `ZLEXCOUNT` counts range.
> **Pattern**: `ZRANGEBYLEX key min max` with `[` inclusive / `(` exclusive; `-`/`+` open bounds.
> **Result**: Members or count. ([Redis][15])

---

#### Hash Index — Hashes (p. 28)

**Pattern.** Use field→value mappings for exact/pattern matches; enumerate with `HSCAN` (cursor) + `MATCH`.

```python
# Index by attributes inside a hash bucket
r.execute_command("HSET", "user:idx:email", "ana@example.com", "user:42")
cursor, rows = r.execute_command("HSCAN", "user:idx:email", 0, "MATCH", "ana*")
print(cursor, rows)
```

> **Sidenote**
>
> [Link: HSCAN](https://redis.io/docs/latest/commands/hscan/)
> **Command**: `HSCAN` → Incremental iteration with optional `MATCH`/`COUNT`.
> **Pattern**: `HSCAN key cursor [MATCH pattern] [COUNT n] [NOVALUES]`
> **Result**: `[cursor, [field, value, ...]]`. ([Redis][3])

---

#### Bitmap Index — Strings as Bitmaps (p. 29)

**Pattern.** Use a string as a bit array; set/query bits; aggregate with bitwise ops; count with `BITCOUNT`. Ideal for Booleans or low‑cardinality attributes.

```python
# Mark user IDs that have "feature X" (bit = user_id)
r.execute_command("SETBIT", "idx:featureX", 42, 1)
has = r.execute_command("GETBIT", "idx:featureX", 42)
total = r.execute_command("BITCOUNT", "idx:featureX")
print(has, total)

# Combine indexes
r.execute_command("BITOP", "AND", "idx:both", "idx:featureX", "idx:featureY")
print("OK - intersected")
```

> **Sidenote**
>
> [Link: SETBIT](https://redis.io/docs/latest/commands/setbit/) • [Link: GETBIT](https://redis.io/docs/latest/commands/getbit/) • [Link: BITCOUNT](https://redis.io/docs/latest/commands/bitcount/) • [Link: BITOP](https://redis.io/docs/latest/commands/bitop/)
> **Command**: Bitmap set/get/popcount/bitwise ops.
> **Pattern**: `SETBIT key offset 0|1` • `GETBIT key offset` • `BITCOUNT key [start end [BYTE|BIT]]` • `BITOP op dest k1 k2..`
> **Result**: Integers/OK. ([Redis][16])

---

#### Full‑Text & JSON Indexing — Redis Query Engine (RediSearch) (p. 30)

**Pattern.** Create inverted indexes over Hashes or JSON documents; support text search, numeric/geo filters, prefixes, field scoping, highlighting, aggregations, and more.

```python
# Store JSON document
r.execute_command("JSON.SET", "item:1", "$", '{"name":"Noise-cancelling headphones","price":99.98,"loc":[-122.42,37.77]}')

# Create index on JSON (name text, price numeric, geo point)
r.execute_command(
    "FT.CREATE", "idx:items",
    "ON", "JSON",
    "SCHEMA",
    "$.name", "AS", "name", "TEXT",
    "$.price", "AS", "price", "NUMERIC",
    "$.loc", "AS", "loc", "GEO"
)

# Search with filters
r.execute_command("FT.SEARCH", "idx:items", "@name:(headphones) @price:[50 150]")
print("FT.SEARCH ok")
```

> **Sidenote**
>
> [Link: FT.CREATE](https://redis.io/docs/latest/commands/ft.create/)
> **Command**: `FT.CREATE` → Define index over Hash/JSON (e.g., `ON JSON` + JSONPath schema).
> **Pattern**: `FT.CREATE idx ON JSON SCHEMA $.path AS attr TYPE [OPTIONS]`
> **Result**: `OK`. *(Requires Redis with the Query Engine; Redis Stack bundles it.)* ([Redis][17])

> **Sidenote**
>
> [Link: FT.SEARCH](https://redis.io/docs/latest/commands/ft.search/)
> **Command**: `FT.SEARCH` → Query text/numeric/tag/geo, with paging, RETURN, HIGHLIGHT, etc.
> **Pattern**: `FT.SEARCH idx query [FILTERS...] [LIMIT offset n] [RETURN n fields...]`
> **Result**: `[total, id, fields..., id, fields...]`. ([Redis][18])

> **Sidenote**
>
> [Link: FT.AGGREGATE](https://redis.io/docs/latest/commands/ft.aggregate/)
> **Command**: `FT.AGGREGATE` → Pipeline-style group‑by, reduce, sort, apply.
> **Pattern**: `FT.AGGREGATE idx query [GROUPBY ... REDUCE ... APPLY ... SORTBY ... LIMIT ...]`
> **Result**: Rows of aggregate results. ([Redis][19])

---

#### Geo Filtering (part of Querying, p. 30)

**Pattern.** Use Geo indexes for proximity queries.

```python
# Native GEO index
r.execute_command("GEOADD", "geo:stores", -122.42, 37.77, "SFO", -118.24, 34.05, "LAX")

# Bounding search around lon/lat
res = r.execute_command("GEOSEARCH", "geo:stores", "FROMLONLAT", -122.4, 37.8, "BYRADIUS", 50, "MI", "WITHDIST")
print(res)
```

> **Sidenote**
>
> [Link: GEOSEARCH](https://redis.io/docs/latest/commands/geosearch/)
> **Command**: `GEOSEARCH` → Radius/box queries; return distance/coords/hash.
> **Pattern**: `GEOSEARCH key FROMLONLAT lon lat BYRADIUS r M|KM|FT|MI [WITHDIST] [WITHCOORD]`
> **Result**: Members with optional distances/coords. ([Redis][20])

---

## Command Reference Snippets (selected)

```python
# Basic strings
r.execute_command("SET", "k", "v"); r.execute_command("GET", "k")
print("OK - string set/get")

# Multi-get to collapse N+1
r.execute_command("MGET", "user:1", "user:2", "user:3")
print("OK - batch read")
```

> **Sidenote**
>
> [Link: GET](https://redis.io/docs/latest/commands/get/) • [Link: SET](https://redis.io/docs/latest/commands/set/) • [Link: MGET](https://redis.io/docs/latest/commands/mget/)
> **Command**: String get/set and batched retrieval.
> **Pattern**: `SET key value [EX|PX|... ttl]` • `GET key` • `MGET k1 k2 ...`
> **Result**: `OK` / value / array of values. ([Redis][21])

```python
# Rankings by score range (numeric index)
r.execute_command("ZRANGE", "lb:game", "0", "100", "BYSCORE", "WITHSCORES")
print("OK - score window")
```

> **Sidenote**
>
> [Link: ZRANGE](https://redis.io/docs/latest/commands/zrange/)
> **Command**: `ZRANGE` (modern) → ranges by rank, score (`BYSCORE`), or lex (`BYLEX`), with `REV`, `LIMIT`, `WITHSCORES`.
> **Result**: Members/tuples depending on options. ([Redis][14])

```python
# Streams consumer lag
r.execute_command("XPENDING", "events:sales", "g1")
print("OK - group stats")
```

> **Sidenote**
>
> [Link: XPENDING](https://redis.io/docs/latest/commands/xpending/)
> **Command**: Inspect pending entries per group/consumer.
> **Pattern**: `XPENDING key group [start end count [consumer]]`
> **Result**: Summary or detailed entries. ([Redis][13])

---

## Page Map / Provenance

* **Caching** overview, patterns, and pros/cons (pp. 4–10).  &#x20;
* **Session Store** overview, differences, and examples (pp. 12–14). &#x20;
* **Leaderboards** requirements and ZSET fit (pp. 16–17).&#x20;
* **Messaging** with Lists, Sorted‑set priority, Pub/Sub, Streams (pp. 20–23). &#x20;
* **Indexing & Querying** numeric, lexicographic, hash, bitmap, full‑text (pp. 25–30). &#x20;

---


### Sources

- [TTL | Docs - Redis](https://redis.io/docs/latest/commands/ttl/?utm_source=chatgpt.com)
- [Write-behind caching | Docs - Redis](https://redis.io/docs/latest/operate/oss_and_stack/stack-with-enterprise/gears-v1/python/recipes/write-behind/?utm_source=chatgpt.com)
- [HSCAN | Docs - Redis](https://redis.io/docs/latest/commands/hscan/?utm_source=chatgpt.com)
- [GitHub - RedisGears/rgsync: A Write-Behind and Write-Through recipe](https://github.com/RedisGears/rgsync?utm_source=chatgpt.com)
- [What is change data capture (CDC)? - SQL Server](https://learn.microsoft.com/en-us/sql/relational-databases/track-changes/about-change-data-capture-sql-server?view=sql-server-ver17&utm_source=chatgpt.com)
- [Redis persistence | Docs](https://redis.io/docs/latest/operate/oss_and_stack/management/persistence/?utm_source=chatgpt.com)
- [High availability with Redis Sentinel | Docs](https://redis.io/docs/latest/operate/oss_and_stack/management/sentinel/?utm_source=chatgpt.com)
- [Redis sorted sets | Docs](https://redis.io/docs/latest/develop/data-types/sorted-sets/?utm_source=chatgpt.com)
- [ZINCRBY | Docs - Redis](https://redis.io/docs/latest/commands/zincrby/?utm_source=chatgpt.com)
- [BRPOP | Docs - Redis](https://redis.io/docs/latest/commands/brpop/?utm_source=chatgpt.com)
- [ZPOPMIN | Docs - Redis](https://redis.io/docs/latest/commands/zpopmin/?utm_source=chatgpt.com)
- [Redis Pub/sub | Docs](https://redis.io/docs/latest/develop/pubsub/?utm_source=chatgpt.com)
- [Redis Streams | Docs](https://redis.io/docs/latest/develop/data-types/streams/?utm_source=chatgpt.com)
- [ZRANGE | Docs - Redis](https://redis.io/docs/latest/commands/zrange/?utm_source=chatgpt.com)
- [ZRANGEBYLEX | Docs - Redis](https://redis.io/docs/latest/commands/zrangebylex/?utm_source=chatgpt.com)
- [SETBIT | Docs - Redis](https://redis.io/docs/latest/commands/setbit/?utm_source=chatgpt.com)
- [FT.CREATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.create/?utm_source=chatgpt.com)
- [FT.SEARCH | Docs - Redis](https://redis.io/docs/latest/commands/ft.search/?utm_source=chatgpt.com)
- [FT.AGGREGATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.aggregate/?utm_source=chatgpt.com)
- [GEOSEARCH | Docs - Redis](https://redis.io/docs/latest/commands/geosearch/?utm_source=chatgpt.com)
- [Redis Strings | Docs](https://redis.io/docs/latest/develop/data-types/strings/?utm_source=chatgpt.com)


### Original Transcript